## 다운스트림 태스크

### 문서 분류

* 데이터 : 네이버 평점 데이터(train)  
* 분석 목적 : 네이버 평점에 대한 긍정과 부정(이진분류)  
* 분석 성능 기준 : 정확도  

In [1]:
# 졍규 표현식 패키지
import re

# 토크나이저 패키지
from nltk.tokenize import word_tokenize
from ckonlpy.tag import Twitter
from konlpy.tag import Kkma

from tokenizers import Tokenizer
from nltk.tokenize import RegexpTokenizer

# 벡터화 패키지
from sklearn.feature_extraction.text import TfidfVectorizer

# 분류 모델 패키지
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier

!pip install xgboost
from xgboost import XGBClassifier

# 분류 모델 평가
from sklearn.metrics import classification_report

# 데이터 핸들링
import numpy as np
import pandas as pd

# 기타
from tqdm import tqdm # 분석 속도, 시간 확인, 정도 확인

     --------------------------------------- 99.7/99.7 MB 10.1 MB/s eta 0:00:00


In [61]:
train_df = pd.read_csv('./data/ratings_train.csv')

In [62]:
train_df.dropna(inplace=True)

In [63]:
# 1. 데이터 분리 
X_train, y_train = train_df['document'][:100000], train_df['label'][:100000]
X_test, y_test = train_df['document'][100000:] , train_df['label'][100000:]

C:\Users\user\AppData\Local\Temp\ipykernel_11208\4009781535.py:2: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  X_train, y_train = train_df['document'][:100000], train_df['label'][:100000]
C:\Users\user\AppData\Local\Temp\ipykernel_11208\4009781535.py:3: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  X_test, y_test = train_df['document'][100000:] , train_df['label'][100000:]


In [64]:
X_train.shape , X_test.shape

((100000,), (49995,))

사용자 사전 추가

### 사용자 토크나이저 만들기

In [65]:
class UserTokenizers :
    def __init__(self) -> None:
        self.okt = Twitter()
        self.kkma = Kkma()
        self.bpe_tokenizer_pretrained = Tokenizer.from_file('./tokenizer_data/bpe_tokenizer.json')

    @staticmethod
    def whitespaceToken(data : str) -> list :
        token_rs = data.split(' ')
        return token_rs
    
    @staticmethod
    def regexsplitToken(data : str , pat : str = '[\,\.!?\n]') -> list :
        re_rs = re.split(pat, data , maxsplit= 0)
        token_rs = [rs_unit.strip() for rs_unit in re_rs if len(rs_unit.strip()) > 1]
        return token_rs
    
    @staticmethod
    def regexselectToken(data : str , pat : str = '[\w]+') -> list :
        token_rs = RegexpTokenizer(pat).tokenize(data)
        return token_rs
    
    def BPETokenizer(self, data : str ) -> list :
        token_rs = self.bpe_tokenizer_pretrained.encode(data).tokens
        return token_rs
    
    #한글과 영어를 같이 사용
    def tokenizingKorEng(self, data :str ) -> list :
        kor_re = re.findall('[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+', data)
        kor_str = ' '.join(kor_re)

        eng_re = re.findall('[a-zA-Z]+', data)
        eng_str = ' '.join(eng_re)

        kor_rs = self.okt.morphs(kor_str)
        eng_rs = word_tokenize(eng_str)

        token_rs = kor_rs + eng_rs

        return token_rs
    
    def konlpyNounsTokenizer(self, data:str) -> list :
        token_rs = self.kkma.nouns(data)
        return token_rs
    

In [66]:
ut_cls = UserTokenizers()

c:\ProgramData\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


사용자 토크나이저 적용 및 확인

In [67]:
X_train[5]

'막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.'

In [68]:
ut_cls.tokenizingKorEng(X_train[5])

['막',
 '걸음',
 '마',
 '뗀',
 '세부',
 '터',
 '초등학교',
 '학년',
 '생인',
 '살용',
 '영화',
 'ㅋㅋㅋ',
 '별반',
 '개도',
 '아까',
 '움']

In [69]:
ut_cls.konlpyNounsTokenizer(X_train[5])

['걸음마',
 '3',
 '3세',
 '세',
 '초등학교',
 '1',
 '1학년생인',
 '학년',
 '생인',
 '8',
 '8살용영화',
 '살',
 '용',
 '영화',
 '별',
 '별반개',
 '반개']

In [70]:
ut_cls.BPETokenizer(X_train[5])

['막</w>',
 '걸',
 '음',
 '마</w>',
 '<unk>',
 '3',
 '세',
 '부터</w>',
 '초등학교</w>',
 '1',
 '학년',
 '생',
 '인</w>',
 '8',
 '살',
 '용',
 '영화</w>',
 '.</w>',
 'ㅋㅋㅋ</w>',
 '.</w>',
 '.</w>',
 '.</w>',
 '별',
 '반개도</w>',
 '아까움</w>',
 '.</w>']

불용어 적용하기

In [71]:
stopwords = ['의', '가', '은', '는', '이', '을', '를', '으로', '에', '와', '과']

벡터화 하기

In [72]:
# tfidf_vect = TfidfVectorizer(tokenizer=ut_cls.BPETokenizer , stop_words=stopwords, ngram_range=(1,2))
tfidf_vect = TfidfVectorizer(tokenizer=ut_cls.BPETokenizer , stop_words=stopwords)

In [73]:
tfidf_vect.fit(X_train)

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['가</w>', '과</w>', '는</w>', '를</w>', '에</w>', '와</w>', '으로</w>', '은</w>', '을</w>', '의</w>', '이</w>'] not in stop_words.
  warnings.warn(


TfidfVectorizer(stop_words=['의', '가', '은', '는', '이', '을', '를', '으로', '에', '와',
                            '과'],
                tokenizer=<bound method UserTokenizers.BPETokenizer of <__main__.UserTokenizers object at 0x00000275BCED3AF0>>)

In [49]:
tfidf_vect.get_feature_names_out()

array(['!</w>', '"</w>', '%</w>', ..., '힘든</w>', '힘들', '힘들었'],
      dtype=object)

In [129]:
class TransformVect :
    def __init__(self, vec_model) -> None:
        # 서비스로 개발을 한다면, 저장된 vec_model을 로딩하는 부분으로 변경
        self.vec_model = vec_model
    
    def transVect_run(self, chunk_size : int = 500 , data : pd.DataFrame = None ) -> np.array :
        
        data_len = len(data) 

        for st_idx in tqdm(range(0, data_len, chunk_size)):
            tmp_data = data[st_idx:st_idx+chunk_size]

            if st_idx == 0:
                vec_arr = self.vec_model.transform(tmp_data).toarray()
                # print(f"num1 : {vec_arr.shape} || dim : {vec_arr.ndim}")

            else :
                tmp_data_arr = self.vec_model.transform(tmp_data).toarray()
                # print(f"num2 : {tmp_data_arr.shape} || dim : {tmp_data_arr.ndim}")

                vec_arr = np.concatenate([vec_arr, tmp_data_arr], 0)                
                # print(f"num2 more : tmp_data_arr : {tmp_data_arr.shape} || vec_arr : {vec_arr.shape}")

        return vec_arr



In [130]:
tfv_cls = TransformVect(tfidf_vect)

In [123]:
# X_train, y_train = X_train[:500],  y_train[:500]
# X_test , y_test = X_test[:100] , y_test[:100]

In [124]:
y_train.sum() , y_train.count() , y_test.sum() , y_test.count()

(49885, 100000, 24940, 49995)

In [125]:
x_train = tfv_cls.transVect_run(chunk_size=100, data=X_train)

  0%|          | 0/1000 [00:00<?, ?it/s]C:\Users\user\AppData\Local\Temp\ipykernel_11208\3731986954.py:11: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  tmp_data = data[st_idx:st_idx+chunk_size]
  1%|          | 6/1000 [00:00<00:16, 59.51it/s]

num1 : (100, 9858) || dim : 2
num2 : (100, 9858) || dim : 2
num2 more : tmp_data_arr : (100, 9858) || vec_arr : (200, 9858)
num2 : (100, 9858) || dim : 2
num2 more : tmp_data_arr : (100, 9858) || vec_arr : (300, 9858)
num2 : (100, 9858) || dim : 2
num2 more : tmp_data_arr : (100, 9858) || vec_arr : (400, 9858)
num2 : (100, 9858) || dim : 2
num2 more : tmp_data_arr : (100, 9858) || vec_arr : (500, 9858)
num2 : (100, 9858) || dim : 2
num2 more : tmp_data_arr : (100, 9858) || vec_arr : (600, 9858)
num2 : (100, 9858) || dim : 2
num2 more : tmp_data_arr : (100, 9858) || vec_arr : (700, 9858)
num2 : (100, 9858) || dim : 2
num2 more : tmp_data_arr : (100, 9858) || vec_arr : (800, 9858)
num2 : (100, 9858) || dim : 2
num2 more : tmp_data_arr : (100, 9858) || vec_arr : (900, 9858)
num2 : (100, 9858) || dim : 2
num2 more : tmp_data_arr : (100, 9858) || vec_arr : (1000, 9858)


  1%|          | 12/1000 [00:00<00:23, 41.81it/s]

num2 : (100, 9858) || dim : 2
num2 more : tmp_data_arr : (100, 9858) || vec_arr : (1100, 9858)
num2 : (100, 9858) || dim : 2
num2 more : tmp_data_arr : (100, 9858) || vec_arr : (1200, 9858)
num2 : (100, 9858) || dim : 2
num2 more : tmp_data_arr : (100, 9858) || vec_arr : (1300, 9858)
num2 : (100, 9858) || dim : 2
num2 more : tmp_data_arr : (100, 9858) || vec_arr : (1400, 9858)
num2 : (100, 9858) || dim : 2
num2 more : tmp_data_arr : (100, 9858) || vec_arr : (1500, 9858)
num2 : (100, 9858) || dim : 2
num2 more : tmp_data_arr : (100, 9858) || vec_arr : (1600, 9858)
num2 : (100, 9858) || dim : 2


  2%|▏         | 21/1000 [00:00<00:33, 28.80it/s]

...

num2 more : tmp_data_arr : (100, 9858) || vec_arr : (99800, 9858)
num2 : (100, 9858) || dim : 2


100%|█████████▉| 999/1000 [29:15<00:09,  9.49s/it]

num2 more : tmp_data_arr : (100, 9858) || vec_arr : (99900, 9858)
num2 : (100, 9858) || dim : 2


100%|██████████| 1000/1000 [29:23<00:00,  1.76s/it]

num2 more : tmp_data_arr : (100, 9858) || vec_arr : (100000, 9858)


In [131]:
x_test = tfv_cls.transVect_run(chunk_size=100, data=X_test)

  0%|          | 0/500 [00:00<?, ?it/s]C:\Users\user\AppData\Local\Temp\ipykernel_11208\1455662524.py:11: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  tmp_data = data[st_idx:st_idx+chunk_size]
100%|██████████| 500/500 [04:14<00:00,  1.96it/s]


In [32]:
# x_train = tfidf_vect.transform(X_train).toarray()
# x_test = tfidf_vect.transform(X_test).toarray()

In [132]:
x_train.shape , x_test.shape

((100000, 9858), (49995, 9858))

분류 모델 적용하기

In [133]:
# 고정된 모델 결과
random_seed_num = 42

In [134]:
# 의사결정나무 
clf_decision = DecisionTreeClassifier(random_state=random_seed_num, max_depth=5)
clf_decision.fit(x_train, y_train)

y_train_pred = clf_decision.predict(x_train)
# y_test_pred = clf_decision.predict(x_test)

In [135]:
print(classification_report(y_train, y_train_pred))
# print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.54      0.95      0.69     50115
           1       0.80      0.19      0.31     49885

    accuracy                           0.57    100000
   macro avg       0.67      0.57      0.50    100000
weighted avg       0.67      0.57      0.50    100000



In [136]:
# randomforest 

rfc_decision = RandomForestClassifier(random_state=random_seed_num, max_depth=5)
rfc_decision.fit(x_train, y_train)

y_train_pred = rfc_decision.predict(x_train)
y_test_pred = rfc_decision.predict(x_test)

In [137]:
print(classification_report(y_train, y_train_pred))
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.68      0.85      0.76     50115
           1       0.80      0.60      0.69     49885

    accuracy                           0.73    100000
   macro avg       0.74      0.73      0.72    100000
weighted avg       0.74      0.73      0.72    100000

              precision    recall  f1-score   support

           0       0.68      0.84      0.75     25055
           1       0.79      0.60      0.68     24940

    accuracy                           0.72     49995
   macro avg       0.73      0.72      0.71     49995
weighted avg       0.73      0.72      0.71     49995



In [138]:
# kneighbors 

clf_kneighbors = KNeighborsClassifier(n_neighbors=3 , weights='distance', leaf_size=50)
clf_kneighbors.fit(x_train, y_train)

y_train_pred = clf_kneighbors.predict(x_train)
y_test_pred = clf_kneighbors.predict(x_test)

In [139]:
print(classification_report(y_train, y_train_pred))
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     50115
           1       1.00      1.00      1.00     49885

    accuracy                           1.00    100000
   macro avg       1.00      1.00      1.00    100000
weighted avg       1.00      1.00      1.00    100000

              precision    recall  f1-score   support

           0       0.77      0.76      0.76     25055
           1       0.76      0.77      0.76     24940

    accuracy                           0.76     49995
   macro avg       0.76      0.76      0.76     49995
weighted avg       0.76      0.76      0.76     49995



In [140]:
# 로지스틱 

clf_logistic = LogisticRegression(max_iter=3000, random_state=random_seed_num)
clf_logistic.fit(x_train, y_train)

y_train_pred = clf_logistic.predict(x_train)
y_test_pred = clf_logistic.predict(x_test)

In [141]:
print(classification_report(y_train, y_train_pred))
print(classification_report(y_test, y_test_pred))




              precision    recall  f1-score   support

           0       0.87      0.88      0.88     50115
           1       0.88      0.87      0.88     49885

    accuracy                           0.88    100000
   macro avg       0.88      0.88      0.88    100000
weighted avg       0.88      0.88      0.88    100000

              precision    recall  f1-score   support

           0       0.85      0.85      0.85     25055
           1       0.85      0.84      0.85     24940

    accuracy                           0.85     49995
   macro avg       0.85      0.85      0.85     49995
weighted avg       0.85      0.85      0.85     49995



In [142]:
# xgb 

clf_xgb = XGBClassifier(random_state = random_seed_num, max_depth=5)
clf_xgb.fit(x_train, y_train)

y_train_pred = clf_xgb.predict(x_train)
y_test_pred = clf_xgb.predict(x_test)

In [143]:
print(classification_report(y_train, y_train_pred))
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.73      0.86      0.79     50115
           1       0.83      0.68      0.75     49885

    accuracy                           0.77    100000
   macro avg       0.78      0.77      0.77    100000
weighted avg       0.78      0.77      0.77    100000

              precision    recall  f1-score   support

           0       0.71      0.84      0.77     25055
           1       0.80      0.66      0.72     24940

    accuracy                           0.75     49995
   macro avg       0.76      0.75      0.75     49995
weighted avg       0.76      0.75      0.75     49995



In [144]:
# 앙상블 모델

# voting 방식 
## soft : 각 모델의 결과가 특정 라벨이라고 하는 확률값을 사용하여 결과 도출
## hard : 각 모델의 최종 결과(라벨)에서 다수로 라벨링된 결과를 도출, 0 : 5개 , 1 : 1개 => 0인 라벨 / 보팅 모델의 세부 총수 : 6개 

clf_ensemble = VotingClassifier(estimators=[('logistic', clf_logistic), ('kneighbors', clf_kneighbors), ('xgboost', clf_xgb)], voting = 'soft')
clf_ensemble.fit(x_train, y_train)

y_train_pred = clf_ensemble.predict(x_train)
y_test_pred = clf_ensemble.predict(x_test)



In [145]:
print(classification_report(y_train, y_train_pred))
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99     50115
           1       0.99      0.99      0.99     49885

    accuracy                           0.99    100000
   macro avg       0.99      0.99      0.99    100000
weighted avg       0.99      0.99      0.99    100000

              precision    recall  f1-score   support

           0       0.83      0.84      0.83     25055
           1       0.84      0.83      0.83     24940

    accuracy                           0.83     49995
   macro avg       0.83      0.83      0.83     49995
weighted avg       0.83      0.83      0.83     49995



## 메모리 에러가 나는 경우의 대처방법

1. 하드웨어 변경 
    - 메모리를 up 
    - 메모리가 넉넉한 서버로 이동하여 분석

2. 코드로 제어
    - chunk_size : 분석을 분할한다.
    - 모델 제어 : word2vec / countVect / tfidfVect < = 통계기반으로 메모리 사용이 많을 수 있음 
        => vocab 사이즈를 제어
        => 피처 추출 : 전체 토큰화된 텍스트 데이터 => 명사만 분석

- 에러가 나는 케이스
  - numpy : index 값이 특정값 이상(int64)인 경우 sorting 등에서 에러 발생
